In [1]:
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from functools import partial

%matplotlib inline

print('cuda:', torch.cuda.is_available())

cuda: True


### bottleneck feature extraction

### resnet50

In [2]:
def extract_features(model, dataloader, epochs):
    model = model.cuda()
    target = []
    scores = []
    for epoch in range(epochs):
        counter = 0
        for data in dataloader:
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            outputs = model(inputs)
            target += [np.squeeze(labels.data.cpu().numpy())]
            scores += [np.squeeze(outputs.data.cpu().numpy())]
            counter += 1    
        print('epoch {} finished.'.format(epoch))
    model = model.cpu()
    scores = pd.DataFrame(np.array(scores))
    scores['target'] = target
    print('data shape:', scores.shape)
    return scores

In [3]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.RandomCrop(200)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=20)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)

In [ ]:
trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/valid/', transform=transforms)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=1, shuffle=False, num_workers=1)
testset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/test/', transform=transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))

train_resnet50 = extract_features(resnet50, train_loader, 6)
train_reset.to_csv('../../data/plant-seed/data/features/train_resnet_50.csv', index=False)
del train_resnet50

valid_resnet50 = extract_features(resnet50, valid_loader, 4)
valid_reset.to_csv('../../data/plant-seed/data/features/valid_resnet_50.csv', index=False)
del valid_resnet50

test_resnet50 = extract_features(resnet50, test_loader, 4)
test_reset.to_csv('../../data/plant-seed/data/features/test_resnet_50.csv', index=False)
del test_resnet50

del resnet50

### inception-v3

In [ ]:
def extract_features(model, dataloader, epochs):
    model = model.cuda()
    target = []
    scores = []
    for epoch in range(epochs):
        counter = 0
        for data in dataloader:
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            outputs = model(inputs)[0]
            target += [np.squeeze(labels.data.cpu().numpy())]
            scores += [np.squeeze(outputs.data.cpu().numpy())]
            counter += 1    
        print('epoch {} finished.'.format(epoch))
    model = model.cpu()
    scores = pd.DataFrame(np.array(scores))
    scores['target'] = target
    print('data shape:', scores.shape)
    return scores

In [ ]:
transforms = []
transforms += [torchvision.transforms.Resize(299)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=20)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)

In [ ]:
trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/train/', transform=transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/valid/', transform=transforms)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=1, shuffle=False, num_workers=1)
testset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/data/test/', transform=transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
inception = torchvision.models.inception_v3(pretrained=True)
inception.fc = torch.nn.Linear(2048, 2048)

train_inception = extract_features(inception, train_loader, 6)
train_inception.to_csv('../../data/plant-seed/data/features/train_inception.csv', index=False)
del train_inception

valid_inception = extract_features(inception, valid_loader, 4)
valid_inception.to_csv('../../data/plant-seed/data/features/valid_inception.csv', index=False)
del valid_inception

test_inception = extract_features(inception, test_loader, 4)
test_inception.to_csv('../../data/plant-seed/data/features/test_inception.csv', index=False)
del test_inception

del inception